In [564]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from mlpp.hawkesnoparam.estim import Estim
import mlpp.pp.hawkes as hk
from whma.metrics import rel_err, rank_corr, mse_K_partial
from whma.cumulants import get_C, get_K
from numba import autojit

In [565]:
d = 10
mus = 0.0005 * np.ones(d)
Alpha_truth = np.ones((d,d)) / 15
Beta = np.ones((d,d))
_, s, _ = np.linalg.svd(Alpha_truth)
print(s.max())
assert s.max() < 1, "Alpha_truth cannot generate a stable Hawkes process"

0.666666666667


In [566]:
hMax = 40
hDelta = .01
from math import log
# the criterion for beta_min and beta_max are designed for the exponential case
beta_min = log(1000) / hMax
beta_max = log(10./9.) / hDelta
print("We have")
print("    beta_min = ",beta_min)
print("    beta_max = ",beta_max)

We have
    beta_min =  0.17269388197455343
    beta_max =  10.536051565782634


In [246]:
kernels = [[hk.HawkesKernelExp(a, b) for (a, b) in zip(a_list, b_list)] for (a_list, b_list) in zip(Alpha_truth, Beta)]
h = hk.Hawkes(kernels=kernels, mus=list(mus))
T_max = 100000000
h.simulate(T_max)
estim = Estim(h, n_threads=8, hDelta=hDelta, hMax=hMax)

# Second order

In [388]:
L = np.array(estim.lam)
C = get_C(estim)

In [537]:
@autojit
def I1_ij(hk,i,j, H=0.):
    res = 0
    u = 0
    Z_i = hk.get_full_process()[i]
    Z_j = hk.get_full_process()[j]
    n_j = len(Z_j)
    for tau in Z_i:
        while u < n_j and Z_j[u] < tau + H:
            u += 1
        res += u
    return res

@autojit
def I1_ij_alternatif(hk,i,j,H=0.):
    res = 0
    Z_i = hk.get_full_process()[i]
    Z_j = hk.get_full_process()[j]
    for tau in Z_i:
        res += len(Z_j[Z_j<tau+H])
    return res

@autojit
def A_ij(hk,i,j,H):
    res = 0
    u = 0
    count = 0
    T_ = hk.time
    Z_i = hk.get_full_process()[i]
    Z_j = hk.get_full_process()[j]
    n_i = len(Z_i)
    n_j = len(Z_j)
    if H >= 0:
        for tau in Z_i:
            while u < n_j and Z_j[u] < tau:
                u += 1
            v = u
            while v < n_j and Z_j[v] < tau + H:
                v += 1
            if v < n_j: 
                count += 1
                res += v-u
    else:
        for tau in Z_i:
            while u < n_j and Z_j[u] <= tau:
                u += 1
            v = u
            while v >= 0 and Z_j[v] > tau + H:
                v -= 1
            if v >= 0:
                count += 1
                res += u-1-v
    res -= (i==j)*count
    if count < n_i:
        res *= n_i * 1. / count
    res /= T_
    return res

@autojit
def B_ijk(hk,i,j,k,H):
    res = 0
    u = 0
    x = 0
    count = 0
    T_ = hk.time
    H_ = abs(H)
    Z_i = hk.get_full_process()[i]
    Z_j = hk.get_full_process()[j]
    Z_k = hk.get_full_process()[k]
    n_i = len(Z_i)
    n_j = len(Z_j)
    n_k = len(Z_k)
    for tau in Z_k:
        # work on Z_i
        while u < n_i and Z_i[u] <= tau:
            u += 1
        v = u
        while v >= 0 and Z_i[v] > tau - H_:
            v -= 1
        # work on Z_j
        while x < n_j and Z_j[x] <= tau:
            x += 1
        y = x
        while y >= 0 and Z_j[y] > tau - H_:
            y -= 1
        # check if this step is admissible
        if y >= 0 and v >= 0:
            count += 1
            res += (u-1-v-(i==k))*(x-1-y-(j==k))
    if count < n_k:
        res *= n_k * 1. / count
    res /= T_
    return res

In [525]:
#print(I1_ij_alternatif(h,4,6,hMax)-I1_ij_alternatif(h,4,6))
#print(h.time*A_ij(h,4,6,hMax))

In [532]:
newC = np.zeros((d,d))
A = np.zeros((d,d))
B = np.zeros((d,d))
T = [x[-1]-x[0] for x in h.get_full_process()]
Tm = h.time
N = np.array([len(x) for x in h.get_full_process()])
for i in range(d):
    for j in range(d):
        #A[i,j] = (I1_ij(h,j,i,hMax) - I1_ij(h,j,i)) / Tm
        A[i,j] = A_ij(h,i,j,hMax)
        B[i,j] = A_ij(h,i,j,-hMax)
newC = B - np.einsum('i,j->ij',L,L)*hMax

In [533]:
from scipy.linalg import inv
R_truth = inv(np.eye(d)-Alpha_truth)
#np.linalg.norm(C)
C_theory = np.dot(R_truth,np.dot(np.diag(estim.lam),R_truth.T))
print(rel_err(C_theory,C))
print(rel_err(C_theory,2*newC+np.diag(L)))
second_cumul = 2*newC+np.diag(L)

0.00854678302238
0.00873680722823


# Third order

In [394]:
K = get_K(estim,L,C_theory,R_truth)

In [560]:
@autojit
def I_ijk(hk,i,j,k,H):
    return (I1_ij(hk,i,k,H) - I1_ij(hk,i,k)) * (I1_ij(hk,j,k,H) - I1_ij(hk,j,k)) / hk.time

@autojit
def J_ijk(hk,i,j,k,H):
    return 2*(i==j)*(I1_ij(hk,i,k,H) - I1_ij(hk,i,k))/hk.time

@autojit
def moment3_ijk(hk,L,C,i,j,k,H):
    res = 0
    res += I_ijk(hk,i,j,k,H)
    res += I_ijk(hk,k,i,j,H)
    res += I_ijk(hk,j,k,i,H)
    res += J_ijk(h,i,j,k,H) + J_ijk(h,k,i,j,H) + J_ijk(h,j,k,i,H)
    res += (i==j)*(i==k)*2*H*L[i]
    return res

def moment3_ijk_bis(hk,A_,F_,L,H):
    res = 0
    T_ = hk.time
    res += F_[i,j,k] - (i==j)*A_[k,i]
    res += F_[j,k,i] - (k==j)*A_[i,k]
    res += F_[k,i,j] - (i==k)*A_[j,i]
    res += (i==j)*(A_[k,i]+A_[i,k])
    res += (i==k)*(A_[j,i]+A_[i,j])
    res += (k==j)*(A_[k,i]+A_[i,k])
    res += (i==j)*(i==k)*2*H*L[i]
    return res

In [561]:
F = np.zeros((d,d,d))
for i in range(d):
    for j in range(d):
        for k in range(d):
            F[i,j,k] = B_ijk(h,i,j,k,hMax)

In [562]:
newK = np.zeros((d,d,d))
for i in range(d):
    print(i)
    for j in range(d):
        for k in range(d):
            #newK[i,j,k] = (I1_ij(h,j,i,hMax) - I1_ij(h,j,i)) * (I1_ij(h,k,i,hMax) - I1_ij(h,k,i)) / (Tm**2)
            #newK[i,j,k] -= (j==k) * (I1_ij(h,j,i,hMax) - I1_ij(h,j,i)) / Tm
            #newK[i,j,k] = moment3_ijk(h,L,C,i,j,k,hMax)
            newK[i,j,k] = moment3_ijk_bis(h,A,F,L,0.01*hMax)
            #newK[i,j,k] = 0.
            
# the following lines correspond to the equations (10)-(13)
newK -= 4*np.einsum('i,j,k->ijk',L,L,L)*hMax**2
newK -= 2*np.einsum('i,jk->ijk',L,second_cumul)*hMax
newK -= 2*np.einsum('j,ik->ijk',L,second_cumul)*hMax
newK -= 2*np.einsum('k,ij->ijk',L,second_cumul)*hMax
#newK -= 2*np.einsum('i,ij,k->ijk',L,np.eye(d),L)*hMax
#newK -= 2*np.einsum('i,ik,j->ijk',L,np.eye(d),L)*hMax
#newK -= 2*np.einsum('j,jk,i->ijk',L,np.eye(d),L)*hMax

0
1
2
3
4
5
6
7
8
9


In [563]:
print(rel_err(K,newK))
print(rel_err(K,np.zeros_like(K)))

0.0312475525976
1.0
